## 키워드 to 문서 NLI 데이터셋 만들기

In [252]:
import pandas as pd

data = pd.read_csv("cluster_five_result_with_keyword.csv")

In [253]:
data = data[['cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4','keyword']]

In [254]:
data

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,keyword
0,35,32,173,267,364,HR
1,27,73,112,192,341,아토피
2,13,28,128,169,175,향수병
3,38,68,65,251,186,본능
4,38,82,119,246,329,상념
...,...,...,...,...,...,...
6763,6,48,90,219,334,제사
6764,28,64,159,201,163,천상병
6765,35,76,143,191,336,핼러윈
6766,9,20,27,70,115,자유학기제


In [11]:
words = data['keyword'].to_list()

In [16]:
len(words)

6768

In [91]:
import random

def get_rand_word(words):
    rand_idx = random.randint(0,len(words)-1)
    return words[rand_idx]

In [31]:
get_rand_word(words)

'우주론'

In [29]:
def get_clusters(data, word):
    
    clusters = []
    
    row = data[data['keyword'] == word]
    clusters.append(row['cluster_0'].values[0])
    clusters.append(row['cluster_1'].values[0])
    clusters.append(row['cluster_2'].values[0])
    clusters.append(row['cluster_3'].values[0])
    clusters.append(row['cluster_4'].values[0])
    
    return clusters

In [32]:
get_clusters(data, '우주론')

[39, 25, 108, 91, 155]

몇 개의 클러스터가 같은지 확인하는 함수

In [71]:
def counts_same_cluster(cluster1, cluster2):
    cnt = 0
    for i in cluster1:
        if i in cluster2:
            cnt += 1
    return cnt

In [72]:
word1 = get_rand_word(words)
word2 = get_rand_word(words)
print(word1, word2)
cluster1 = get_clusters(data, word1)
cluster2 = get_clusters(data, word2)
print(cluster1, cluster2)
counts_same_cluster(cluster1, cluster2)

스트레칭 작업노트
[0, 12, 39, 16, 17] [17, 54, 41, 4, 42]


1

### 일치하는 클러스터 수에 따라 neutral과 contradiction 단어 리스트 구하기

entail키워드에 속하는 모든 키워드에 대해야 일치하는 클러스터가 0개라면 contradiction, 1 ~ 3 개이면 neutral이다.

entailment는 기존의 태그 3개이다.

In [229]:
def get_contradiction_words(data, words, entail_keywords):
    contradiction_words = []
    while len(contradiction_words) != 3:
        rand_word = get_rand_word(words)   
        cluster = get_clusters(data, rand_word)
        
        is_contradict = True
        for keyword in entail_keywords:
            entail_cluster = get_clusters(data, keyword)
            cnt = counts_same_cluster(cluster, entail_cluster)
            if cnt > 1:
                is_contradict = False
        if is_contradict:
            contradiction_words.append(rand_word)
    return contradiction_words

In [245]:
get_contradiction_words(data, words, ['챗봇', 'AI', '미래'])

['돌담길', '조직성과', '맨몸운동']

In [222]:
def get_neutral_words(data, words, entail_keywords):
    neutral_words = []
    while len(neutral_words) != 2:
        rand_word = get_rand_word(words)   
        cluster = get_clusters(data, rand_word)
        is_neutral = True
        for keyword in entail_keywords:
            entail_cluster = get_clusters(data, keyword)
            cnt = counts_same_cluster(cluster, entail_cluster)
            if cnt < 1 or cnt > 3:
                is_neutral = False
        if is_neutral:
            neutral_words.append(rand_word)
    return neutral_words

In [247]:
get_neutral_words(data, words, ['카피라이터','AI', '콘텐츠'])

['브랜드관리', '커리어패스']

### 기존 데이터 읽어오기

In [101]:
import jsonlines

data_list = []

with jsonlines.open("./data/brunch/brunch_dataset_50_long.jsonl") as f:
    for line in f.iter():
        data_list.append(line)

In [102]:
import pandas as pd

data_df = pd.DataFrame(data_list)
data_df

,title,sub_title,body_text,keyword,likes,post_date,post_url,author_id
0,구글을 대체할 AI 챗봇 ChatGPT,브런치의 미래,\n일론머스크와 샘알트만이 설립한 OpenAI에서 ChatGPT이라는 대화형 인공지...,"[챗봇, AI, 미래]",76,2023-01-18,https://brunch.co.kr/@jejung/178,@jejung
1,"AI 와 예술의 공존, 저작권에 대하여",NaN,\nAI 기술이 나날이 발전을 더하며 그동안 산업계에서 주로 활용되었던 기술이 이제...,"[AI, 예술, NFT]",75,2023-01-04,https://brunch.co.kr/@jejung/176,@jejung
2,AI가 자동으로 콘텐츠를 써주는 서비스의 성능은?,"AI 기반의 카피라이터, '뤼튼' 서비스를 써보았다.","\n\n매일 뭐라도 올리려고 소재를 찾다가, 눈에 띄는 문구가 하나 있었다. 이미 ...","[카피라이터, AI, 콘텐츠]",52,2022-11-09,https://brunch.co.kr/@paulie/264,@paulie
3,AI가 만들어준 찰스 1세와 헨리에타 마리아입니다;;;,(22)책 출간: ai그림 만들기,\n오늘 획기적인 사이트를 하나 알고 들어가 봤습니다.\n\nhttps://nove...,"[텀블벅, AI, 독립출판물]",64,2022-10-13,https://brunch.co.kr/@f635a2b84449453/143,@f635a2b84449453
4,브런치북이 클래스101 AI VOD로 재탄생했습니다,브런치북 AI 클래스 프로젝트 수상작 AI VOD 론칭,"\n브런치북이 전자책, 오디오북에 이어 또 한 번의 변신을 거듭합니다. 브런치북 A...","[브런치북, 클래스, AI]",217,2022-10-05,https://brunch.co.kr/@brunch/313,@brunch
...,...,...,...,...,...,...,...,...
23307,업무 인수인계 잘하기,릴레이에서 바톤을 놓친다면,"\n좋은 직장과 안 좋은 직장을 알 수 있는 기준은 참 많지만, 하나를 골라보라면 ...","[업무, 조직, 회사]",55,2016-03-08,https://brunch.co.kr/@lunarshore/37,@lunarshore
23308,"면접자는 모르는 대기업, 중소기업, 외국계 기업의 특징",당신의 미래를 시작할 곳은 어디인가요,\n** 이 문서는 2017년 기준으로 개정된 문서입니다\n** 유튜브 채널의 좋아...,"[면접, 인재, 회사]",118,2016-03-04,https://brunch.co.kr/@hdyoon/3,@hdyoon
23309,남들이 피하는 일을 선택하라,돌파구 노트,\n\n본인에게 일의 선택권이 주어진다면 어떤 업무를 선택하겠는가? 대부분의 사람들...,"[회사, 도전, 일]",83,2016-01-17,https://brunch.co.kr/@breakthrough/26,@breakthrough
23310,일 잘하는 법,돌파구 노트,\n일을 잘한다는 의미는 무엇일까? 간단히 이야기하면 주어진 시간 안에 업무 목표를...,"[일, 회사, 평가]",91,2016-01-13,https://brunch.co.kr/@breakthrough/25,@breakthrough


### 기존 데이터에 neutral/contradiction 단어 추가하기

기존데이터에 각 행을 순회하며 neutral 단어들과 contradiction 단어를 추가한다.

이때는 데이터 프레임이 아닌 json(딕셔너리)의 리스트인 data_list를 활용한다.

In [258]:
for row in data_list:
    entail_keyword = row['keyword']
    contra_words = get_contradiction_words(data, words, entail_keyword)
    row['contradiction'] = contra_words

In [267]:
data_list[10008]

{'title': '연애보다 사랑',
 'sub_title': '멈춘다고 죽지는 않던데',
 'body_text': '\n나는 삼십 대가 되고 나서야 노화만큼 안타까워진 것이 하나 더 늘었다. 사랑이었다.\n\n어릴 적에 난 연애와 사랑을 하나라고 여겼다. 좋아하는 마음으로 시작된 연애는 마음을 깊고 단단하게 만들어주기도 했다.\n\n나는 사랑을 했고, 사랑이 변했고, 사랑이 떠나가는 과정을 여러 번 거쳐가며 이제야 연애라는 단어와 사랑이라는 단어의 정의를 알게 되었다.\n\n그 둘은 결이 같다고 한들 다른 감정을 가진다. 연애는 성적인 매력에 이끌려 서로 좋아하여 사귀는 것이고 사랑은 어떤 사람이나 존재를 몹시 아끼고 귀중히 여기는 마음이다.\n\n연애가 즐겁고 행복했던 열정의 순간이 있었다. 그것은 시간의 지남에 따라 익숙해졌고, 무뎌졌고, 사라졌다. 며칠 전 현관문을 열고 들어가던 중 외롭다는 감정이 느껴졌고, 연애를 해볼까 했지만 결론은 당장은 아니란 것이었다.\n\n내 시간을 소요하고 싶은 누군가가 당장 주변에 있지 않았고, 소개팅을 해야 하나 싶었지만 상대방을 나에게 맞춰 재단하고 따져대는 행동이 불순하고 귀찮아하지 않기로 했다. \n\n나는 연애가 하고 싶은 건 아닌 것 같다. 연애보다 사랑이 하고 싶다. 누군가를 몹시 귀하고 아꼈던 지난날의 내가 그리웠다보다. 서로 다른 둘이 만나 사랑이라는 감정으로 서로를 아끼고 귀하게 여겼던 날들을 추억한다. 더 이상 누군가를 그렇게 온전하게 사랑할 수 있을지 의문이 들어 사랑에 대한 마음이 더 애달파졌다.\n\n사람은 고쳐 쓰는 게 아니니 잘 맞는 사람을 만나야 한다는 말에 공감한다. 하지만 처음부터 맞는 사람이 어디 쉽게 있을까. 맞지 않은 사람을 사랑해 노력했던 시간도 나에게는 사랑이었다. 노력해도 소용없다는 이유를 붙이고 어차피 헤어지게 될 거란 살을 붙여 사랑에 소극적인 사람이 되었다. \n\n용기 없음을 인정하고 싶지 않았지만 결국 용기 없는 사람이 되었다. 누군가를 잃는 슬픔에 태연하지 못한 어른

In [268]:
for row in data_list:
    entail_keyword = row['keyword']
    neutral_words = get_neutral_words(data, words, entail_keyword)
    row['neutral'] = neutral_words

KeyboardInterrupt: 

In [272]:
for row in data_list:
    row.pop('neutral', None)

In [273]:
aug_data_df = pd.DataFrame(data_list)
aug_data_df.to_csv('brunch_keyword_NLI_dataset.csv')

In [274]:
aug_data_df

,title,sub_title,body_text,keyword,likes,post_date,post_url,author_id,contradiction
0,구글을 대체할 AI 챗봇 ChatGPT,브런치의 미래,\n일론머스크와 샘알트만이 설립한 OpenAI에서 ChatGPT이라는 대화형 인공지...,"[챗봇, AI, 미래]",76,2023-01-18,https://brunch.co.kr/@jejung/178,@jejung,"[선입견, 신뢰관계, 홈스쿨링]"
1,"AI 와 예술의 공존, 저작권에 대하여",NaN,\nAI 기술이 나날이 발전을 더하며 그동안 산업계에서 주로 활용되었던 기술이 이제...,"[AI, 예술, NFT]",75,2023-01-04,https://brunch.co.kr/@jejung/176,@jejung,"[영화관, 시계, 노예]"
2,AI가 자동으로 콘텐츠를 써주는 서비스의 성능은?,"AI 기반의 카피라이터, '뤼튼' 서비스를 써보았다.","\n\n매일 뭐라도 올리려고 소재를 찾다가, 눈에 띄는 문구가 하나 있었다. 이미 ...","[카피라이터, AI, 콘텐츠]",52,2022-11-09,https://brunch.co.kr/@paulie/264,@paulie,"[쌩얼, 화법, 가족영화]"
3,AI가 만들어준 찰스 1세와 헨리에타 마리아입니다;;;,(22)책 출간: ai그림 만들기,\n오늘 획기적인 사이트를 하나 알고 들어가 봤습니다.\n\nhttps://nove...,"[텀블벅, AI, 독립출판물]",64,2022-10-13,https://brunch.co.kr/@f635a2b84449453/143,@f635a2b84449453,"[아트소믈리에, 기준금리, 보리밥]"
4,브런치북이 클래스101 AI VOD로 재탄생했습니다,브런치북 AI 클래스 프로젝트 수상작 AI VOD 론칭,"\n브런치북이 전자책, 오디오북에 이어 또 한 번의 변신을 거듭합니다. 브런치북 A...","[브런치북, 클래스, AI]",217,2022-10-05,https://brunch.co.kr/@brunch/313,@brunch,"[도서관, 기차, MVP]"
...,...,...,...,...,...,...,...,...,...
23307,업무 인수인계 잘하기,릴레이에서 바톤을 놓친다면,"\n좋은 직장과 안 좋은 직장을 알 수 있는 기준은 참 많지만, 하나를 골라보라면 ...","[업무, 조직, 회사]",55,2016-03-08,https://brunch.co.kr/@lunarshore/37,@lunarshore,"[민음사, 티켓팅, 카이사르]"
23308,"면접자는 모르는 대기업, 중소기업, 외국계 기업의 특징",당신의 미래를 시작할 곳은 어디인가요,\n** 이 문서는 2017년 기준으로 개정된 문서입니다\n** 유튜브 채널의 좋아...,"[면접, 인재, 회사]",118,2016-03-04,https://brunch.co.kr/@hdyoon/3,@hdyoon,"[서울역, 안마기, 콘텐츠기획자]"
23309,남들이 피하는 일을 선택하라,돌파구 노트,\n\n본인에게 일의 선택권이 주어진다면 어떤 업무를 선택하겠는가? 대부분의 사람들...,"[회사, 도전, 일]",83,2016-01-17,https://brunch.co.kr/@breakthrough/26,@breakthrough,"[넷플릭스_스토리텔러, 상대방, 환율]"
23310,일 잘하는 법,돌파구 노트,\n일을 잘한다는 의미는 무엇일까? 간단히 이야기하면 주어진 시간 안에 업무 목표를...,"[일, 회사, 평가]",91,2016-01-13,https://brunch.co.kr/@breakthrough/25,@breakthrough,"[kbs아나운서, 생선, 브랜드전략]"


### contradiction title 추가하기

In [275]:
title_list = aug_data_df['title'].to_list()

In [276]:
title_list[:10]

['구글을 대체할 AI 챗봇 ChatGPT',
 'AI 와 예술의 공존, 저작권에 대하여',
 'AI가 자동으로 콘텐츠를 써주는 서비스의 성능은?',
 'AI가 만들어준 찰스 1세와 헨리에타 마리아입니다;;;',
 '브런치북이 클래스101 AI VOD로 재탄생했습니다',
 '여행',
 '브런치북 AI 클래스 프로젝트 수상작을 소개합니다',
 '[예고] 클래스101, 브런치북 AI 클래스 프로젝트',
 '인공지능과의 전쟁',
 '인공지능은 인간과 사랑을 나눌 수 있을까']

In [277]:
import random

for row in data_list:
    while True:
        max_idx = len(data_list) - 1
        rand_idx = random.randint(0, max_idx)
        if row['title'] != title_list[rand_idx]:
            row['contradiction_title']  = title_list[rand_idx]
            break

In [278]:
data_list[10]

{'title': '프로덕트 역기획 : 틱톡의 추천 알고리즘',
 'sub_title': nan,
 'body_text': '\n이번 프로덕트 역기획 포스트를 작성하기 위해서 말로만 듣던 틱톡을 늦게서야 설치하고 사용해 보았는데, 사용 소감은 이렇다. \'진작 써봤으면 큰일 날 뻔했다.\' 사실 틱톡은 프라이버시 문제로 설치를 꺼리던 앱이었는데, 막상 인스톨을 하고 사용해보니 무슨.. 이런 개미지옥 같은 앱이 다 있나 싶었다. 선호도 정보를 제공한 건 앱 사용 전 관심 주제 확인이 전부였음에도 어떻게 이렇게 재밌는 영상만 쏙쏙 골라 보여줄 수 있는지. 거기다 피드는 내려도 내려도 끝이 없었다. 인스타그램의 무한 스크롤 피드처럼 로딩이 발생되는 것도 아니었다. 농담이 아니라 브레이크도 없이 혼자 피식거리며 15~20분 정도를 피드 감상으로만 보낸 것 같다. 인스타그램을 쓸 때도 스크린 타임 기능은 써본 적이 없었는데, 틱톡은 필수로 제한해 놔야만 할 것 같다는 위기감이 생길 정도였으니.. 말 다했지, 뭐. \n\n틱톡은 이미 잘 알려진 바와 같이 15초~1분 이내의 짧은 영상을 제작, 공유할 수 있는 소셜 네트워크 서비스이다. 한국에서는 아무 노래 챌린지로도 잘 알려져 있고, 미국 및 일본 등지에서도 10대들 사이에서 많은 인기를 끌고 있다. 비교적 짧은 시간 내 급성장한 서비스로 중국과 글로벌 서비스의 월간 사용자 수가 15억 명에 달할 정도라 하니 그 인기를 짐작할 만하다. 놀랍게도 틱톡이라는 서비스가 출시된 지는 약 4년 정도밖에 되지 않았다. 틱톡은 어떻게 단기간 내 이렇게 많은 사용자를 끌어모을 수 있었을까? 그리고 어떤 방식으로 돈을 벌고 있을까?\n\n틱톡의 비즈니스 모델은?\n\n틱톡은 주로 광고와 인앱 구매로 돈을 벌고 있기 때문에 최대한 많은 사용자들을 유입시키고 그들의 체류 시간을 늘리는 것을 목표로 삼고 있다. 전 세계의 10대들 사이에서 무섭게 유행으로 번진 앱이기는 하지만, 구매력이 부족한 미성년자를 유료 고객으로 전환시키는 것은 어려운

In [279]:
aug_data_df = pd.DataFrame(data_list)
aug_data_df

,title,sub_title,body_text,keyword,likes,post_date,post_url,author_id,contradiction,contradiction_title
0,구글을 대체할 AI 챗봇 ChatGPT,브런치의 미래,\n일론머스크와 샘알트만이 설립한 OpenAI에서 ChatGPT이라는 대화형 인공지...,"[챗봇, AI, 미래]",76,2023-01-18,https://brunch.co.kr/@jejung/178,@jejung,"[선입견, 신뢰관계, 홈스쿨링]",사랑하는 사람과 잘 싸우기
1,"AI 와 예술의 공존, 저작권에 대하여",NaN,\nAI 기술이 나날이 발전을 더하며 그동안 산업계에서 주로 활용되었던 기술이 이제...,"[AI, 예술, NFT]",75,2023-01-04,https://brunch.co.kr/@jejung/176,@jejung,"[영화관, 시계, 노예]",가끔은 대충 사는 것도
2,AI가 자동으로 콘텐츠를 써주는 서비스의 성능은?,"AI 기반의 카피라이터, '뤼튼' 서비스를 써보았다.","\n\n매일 뭐라도 올리려고 소재를 찾다가, 눈에 띄는 문구가 하나 있었다. 이미 ...","[카피라이터, AI, 콘텐츠]",52,2022-11-09,https://brunch.co.kr/@paulie/264,@paulie,"[쌩얼, 화법, 가족영화]",추석과의 추억
3,AI가 만들어준 찰스 1세와 헨리에타 마리아입니다;;;,(22)책 출간: ai그림 만들기,\n오늘 획기적인 사이트를 하나 알고 들어가 봤습니다.\n\nhttps://nove...,"[텀블벅, AI, 독립출판물]",64,2022-10-13,https://brunch.co.kr/@f635a2b84449453/143,@f635a2b84449453,"[아트소믈리에, 기준금리, 보리밥]",4년 8개월의 백수생활
4,브런치북이 클래스101 AI VOD로 재탄생했습니다,브런치북 AI 클래스 프로젝트 수상작 AI VOD 론칭,"\n브런치북이 전자책, 오디오북에 이어 또 한 번의 변신을 거듭합니다. 브런치북 A...","[브런치북, 클래스, AI]",217,2022-10-05,https://brunch.co.kr/@brunch/313,@brunch,"[도서관, 기차, MVP]","엄마의 공부, 방임인지 모범인지는 더 두고 볼 일이다."
...,...,...,...,...,...,...,...,...,...,...
23307,업무 인수인계 잘하기,릴레이에서 바톤을 놓친다면,"\n좋은 직장과 안 좋은 직장을 알 수 있는 기준은 참 많지만, 하나를 골라보라면 ...","[업무, 조직, 회사]",55,2016-03-08,https://brunch.co.kr/@lunarshore/37,@lunarshore,"[민음사, 티켓팅, 카이사르]",미리 보는 UI 디자인 툴 전쟁
23308,"면접자는 모르는 대기업, 중소기업, 외국계 기업의 특징",당신의 미래를 시작할 곳은 어디인가요,\n** 이 문서는 2017년 기준으로 개정된 문서입니다\n** 유튜브 채널의 좋아...,"[면접, 인재, 회사]",118,2016-03-04,https://brunch.co.kr/@hdyoon/3,@hdyoon,"[서울역, 안마기, 콘텐츠기획자]",나갈 때 퇴사 메일 쓰지 말 것
23309,남들이 피하는 일을 선택하라,돌파구 노트,\n\n본인에게 일의 선택권이 주어진다면 어떤 업무를 선택하겠는가? 대부분의 사람들...,"[회사, 도전, 일]",83,2016-01-17,https://brunch.co.kr/@breakthrough/26,@breakthrough,"[넷플릭스_스토리텔러, 상대방, 환율]",개털려봐야 비로소 깨닫는 삶.(feat. 일본북알프스)
23310,일 잘하는 법,돌파구 노트,\n일을 잘한다는 의미는 무엇일까? 간단히 이야기하면 주어진 시간 안에 업무 목표를...,"[일, 회사, 평가]",91,2016-01-13,https://brunch.co.kr/@breakthrough/25,@breakthrough,"[kbs아나운서, 생선, 브랜드전략]",읽는 자가 배운 자를 이긴다


In [280]:
aug_data_df.rename(columns={"contradiction": "contradiction_words"}, inplace=True)

In [281]:
aug_data_df.rename(columns={"keyword": "keywords"}, inplace=True)

In [282]:
aug_data_df

,title,sub_title,body_text,keywords,likes,post_date,post_url,author_id,contradiction_words,contradiction_title
0,구글을 대체할 AI 챗봇 ChatGPT,브런치의 미래,\n일론머스크와 샘알트만이 설립한 OpenAI에서 ChatGPT이라는 대화형 인공지...,"[챗봇, AI, 미래]",76,2023-01-18,https://brunch.co.kr/@jejung/178,@jejung,"[선입견, 신뢰관계, 홈스쿨링]",사랑하는 사람과 잘 싸우기
1,"AI 와 예술의 공존, 저작권에 대하여",NaN,\nAI 기술이 나날이 발전을 더하며 그동안 산업계에서 주로 활용되었던 기술이 이제...,"[AI, 예술, NFT]",75,2023-01-04,https://brunch.co.kr/@jejung/176,@jejung,"[영화관, 시계, 노예]",가끔은 대충 사는 것도
2,AI가 자동으로 콘텐츠를 써주는 서비스의 성능은?,"AI 기반의 카피라이터, '뤼튼' 서비스를 써보았다.","\n\n매일 뭐라도 올리려고 소재를 찾다가, 눈에 띄는 문구가 하나 있었다. 이미 ...","[카피라이터, AI, 콘텐츠]",52,2022-11-09,https://brunch.co.kr/@paulie/264,@paulie,"[쌩얼, 화법, 가족영화]",추석과의 추억
3,AI가 만들어준 찰스 1세와 헨리에타 마리아입니다;;;,(22)책 출간: ai그림 만들기,\n오늘 획기적인 사이트를 하나 알고 들어가 봤습니다.\n\nhttps://nove...,"[텀블벅, AI, 독립출판물]",64,2022-10-13,https://brunch.co.kr/@f635a2b84449453/143,@f635a2b84449453,"[아트소믈리에, 기준금리, 보리밥]",4년 8개월의 백수생활
4,브런치북이 클래스101 AI VOD로 재탄생했습니다,브런치북 AI 클래스 프로젝트 수상작 AI VOD 론칭,"\n브런치북이 전자책, 오디오북에 이어 또 한 번의 변신을 거듭합니다. 브런치북 A...","[브런치북, 클래스, AI]",217,2022-10-05,https://brunch.co.kr/@brunch/313,@brunch,"[도서관, 기차, MVP]","엄마의 공부, 방임인지 모범인지는 더 두고 볼 일이다."
...,...,...,...,...,...,...,...,...,...,...
23307,업무 인수인계 잘하기,릴레이에서 바톤을 놓친다면,"\n좋은 직장과 안 좋은 직장을 알 수 있는 기준은 참 많지만, 하나를 골라보라면 ...","[업무, 조직, 회사]",55,2016-03-08,https://brunch.co.kr/@lunarshore/37,@lunarshore,"[민음사, 티켓팅, 카이사르]",미리 보는 UI 디자인 툴 전쟁
23308,"면접자는 모르는 대기업, 중소기업, 외국계 기업의 특징",당신의 미래를 시작할 곳은 어디인가요,\n** 이 문서는 2017년 기준으로 개정된 문서입니다\n** 유튜브 채널의 좋아...,"[면접, 인재, 회사]",118,2016-03-04,https://brunch.co.kr/@hdyoon/3,@hdyoon,"[서울역, 안마기, 콘텐츠기획자]",나갈 때 퇴사 메일 쓰지 말 것
23309,남들이 피하는 일을 선택하라,돌파구 노트,\n\n본인에게 일의 선택권이 주어진다면 어떤 업무를 선택하겠는가? 대부분의 사람들...,"[회사, 도전, 일]",83,2016-01-17,https://brunch.co.kr/@breakthrough/26,@breakthrough,"[넷플릭스_스토리텔러, 상대방, 환율]",개털려봐야 비로소 깨닫는 삶.(feat. 일본북알프스)
23310,일 잘하는 법,돌파구 노트,\n일을 잘한다는 의미는 무엇일까? 간단히 이야기하면 주어진 시간 안에 업무 목표를...,"[일, 회사, 평가]",91,2016-01-13,https://brunch.co.kr/@breakthrough/25,@breakthrough,"[kbs아나운서, 생선, 브랜드전략]",읽는 자가 배운 자를 이긴다


In [283]:
aug_json_data = aug_data_df.to_dict('records')

In [285]:
aug_json_data[0]

{'title': '구글을 대체할 AI 챗봇 ChatGPT',
 'sub_title': '브런치의 미래\xa0',
 'body_text': '\n일론머스크와 샘알트만이 설립한 OpenAI에서 ChatGPT이라는 대화형 인공지능 챗봇을 선보였습니다. ChatGPT은 사용자와 주고받는 대화에서 질문에 답하도록 설계되어 있습니다. GPT는 질문에 답하는 것은 물론 정확하지 않은 전제에 대해 이의를 제기하기도 하고 부적절한 요청을 거부하는 등 마치 인격을 가진 인간과의 대화 구현이 가능합니다. \n\n이러한 GPT가 주목받게 된 것은 그 기능 자체의 뛰어남도 있지만 글로벌 기업인 구글의 모회사 CEO가 직접 언급하며 급속도로 기사화되었습니다. 구글에서 GPT를 경계하는 이유는 장기적으로 검색 엔진을 대체할 수 있을 것이라 염려 때문입니다. 현재의 검색 엔진은 사용자가 직접 검색 대상을 문법에 맞게 지시 내려야 합니다. 그러나 GPT는 대화형이기 때문에 누구나 손쉽게 이용할 수 있으며 더 다양한 정보를 (데이터)를 확보할 수 있습니다. \n\n제가 GPT에 관심을 갖게 된 것은 바로 GPT로 뛰어난 글을 작성할 수 있다는 부분 때문이었습니다. 실제로 해외 사용자들의 후기를  통해 GPT를 이용해 소설과 시를 썼다는 후기들을 손쉽게 볼 수 있습니다. \n*GPT는 영어로 커뮤니케이션하는 것이 더 원활합니다. 한국어도 지원하나 정보가 적으며 구동이 느립니다.\n\n캐나다 언론인이자 임상 심리학자이자인 조던피터슨(Jordan Bernt Peterson)은 GPT의 위험성에 대해 이야기하였습니다. 그의 날카로운 지적과 더불어 해당 영상을 본 사람들의 반응 또한 높은 통찰을 보여주고 있습니다. (자세한 내용은 아래 연결된 링크에서 확인 가능합니다)\n\n과거 인터넷의 발달로 우리는 편리한 생활을 누리게 되었지만 그만큼 축소된 부분도 존재합니다. 그 부분은 바로 생각에 대한 영역입니다. 세계적인 경영컨설턴트이자 IT 미래학자인 니콜라스 카가 그의 저서 <생각하지 않는 사람들>에서

In [286]:
import json    
    
with open("brunch_dataset_words_title_NLI.jsonl","w", encoding="UTF-8") as f:
    for line in aug_json_data:
        f.writelines(json.dumps(line, ensure_ascii=False))
        f.write("\n")

### train/vaild/test 나누기

In [287]:
random.shuffle(aug_json_data)

In [288]:
aug_json_data[0]

{'title': '건물주 이 팀장이 퇴사하지 못하는 신박한 이유',
 'sub_title': nan,
 'body_text': '\n\n 조물주 위에 건물주. 그 표현이 방송에서도 보이기 시작하던 시기에 같이 근무했던 이 팀장은 바로 그 \'건물주\'였다. 목 좋은 요지에 상가 두 개를 소유했고, 세 개째를 지어 올리는 중이었다. \n \n 외부에서 만났다면 롤모델로 삼을만한 인물이었다. \n 그 옛날 N 잡러의 삶을 살았다. 회사 퇴근과 동시에 다른 일을 시작했고 집에 들어가지 못한 채  다시 출근한 적도 있다 했다. 밖에서 밥을 사 먹을 때 5천 원 넘는 것을 먹기 시작한 지 얼마 안 됐고 EBS 프로그램으로 해외여행을 대신했다. \n 둘째가 태어난 지 백일도 안돼 갑자기 집주인이 집을 비우라 해서 홧김에 다 쓰러져 가는 집을 샀는데 곧바로 재개발 지역에 포함되었다. 처음으로 거금을 손에 쥔 그는 부동산과 주식에 투자를 했고 연달아 성공했다. \'사람이 아무리 잠 안 자고 일을 해도 돈이 돈 버는 속도를 따라가지 못하더라\'며 오래전 경험에서 진리를 깨우쳤다.\n 상가에서 나오는 월세가 주 수입원이며 주가도 그가 들어가기만 하면 오른다 했다. 오죽하면 아침마다 옆 부서 부장이 우리 팀으로 출근해서  \'오늘은 뭐 사야 되는데?"하고 물었을까. 지금 대다수의 직장인들이 꿈꾸는 그것, \'파이프라인 구축\'을 그 당시에 끝내 놓은 그는 재테크의 선구자였다.\n  \n 그런데 문제는, 나는 그와 한 팀에서 근무를 했다는 거다. 회사 업무보다 중요한 일들이 그에게 너무 많았다. 상가 관리와 건축, 주식 시장까지. 어지간한 사람이면 한 가지도 벅찰 일은 그는 배우자에게도 맡기지 않고 혼자서 다 처리했다. 사무실에 앉아 핸드폰으로 공사현장 CCTV를 구석구석 감시하는 모습이 신기했다. 그 와중에도 그의 핸드폰은 쉴 새 없이 울렸다. 다른 전화를 받을 때와는 사뭇 느낌이 다른 말투로 "몇 평, 어떤 타입을 원하시는 거죠?" 하며 전화를 받기 위해 회의실로 들어갔다. 

In [289]:
len(aug_json_data)

23312

8 : 1 : 1 로 나누기

In [290]:
train_ratio = int(len(aug_json_data)*0.8)
train_ratio

18649

In [293]:
valid_ratio = int(len(aug_json_data)*0.9)
valid_ratio

20980

In [304]:
train_data = aug_json_data[:train_ratio]
valid_data = aug_json_data[train_ratio:valid_ratio]
test_data = aug_json_data[valid_ratio:]

In [305]:
len(train_data)

18649

In [306]:
len(valid_data)

2331

In [307]:
len(test_data)

2332

In [308]:
len(train_data) + len(valid_data) + len(test_data)

23312

In [309]:
import json    
    
with open("brunch_NLI_train.jsonl","w", encoding="UTF-8") as f:
    for line in train_data:
        f.writelines(json.dumps(line, ensure_ascii=False))
        f.write("\n")
        
with open("brunch_NLI_valid.jsonl","w", encoding="UTF-8") as f:
    for line in train_data:
        f.writelines(json.dumps(line, ensure_ascii=False))
        f.write("\n")
        
with open("brunch_NLI_test.jsonl","w", encoding="UTF-8") as f:
    for line in train_data:
        f.writelines(json.dumps(line, ensure_ascii=False))
        f.write("\n")